In questo notebook (che corrisponde al capito 3 di HA_LLM) viene approfondito il funzionamento di un LLM.<br>
Nel precedente abbiamo visto il funzionamento del tokenizer

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-4-mini-instruct",
    torch_dtype="auto",
    trust_remote_code="True",
)
tokenizer = AutoTokenizer.from_pretrained( "microsoft/Phi-4-mini-instruct")

# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)


Loading checkpoint shards: 100%|██████████████████| 2/2 [00:00<00:00, 72.66it/s]
Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [5]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt)

print(output[0]['generated_text'])


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 The email should be in all lowercase letters and include at least 3 placeholders such as [garden name], [plant type], and [garden location]. Dear sarah,

i hope this email finds you well. i am writing to sincerely apologize for the


In [7]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(200064, 3072, padding_idx=199999)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=5120, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_features=20006

Per capire come avviene la scelta del next token è necessario approfondire cosa avviene a livello di llm_head.<br>
E' a questo livello che l'output del modello viene convertito in una probability distribution per ciascun token che compone il vocabolary.

E' quindo fondamentale la scelta della <b>DECODING STRATEGY</b><br>
Possibili decoding strategies sono:
- scegliere sempre il token con max prob (greedy decoding)
- scegliere sulla base della probability distribution

In [15]:
prompt = "The capital of France is"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Tokenize the input prompt
input_ids = input_ids.to("cpu")

# Get the output of the model before the lm_head
model_output = model.model(input_ids)

print(model_output[0].shape)

# Get the output of the lm_head
lm_head_output = model.lm_head(model_output[0])

lm_head_output.shape

torch.Size([1, 5, 3072])


torch.Size([1, 5, 200064])

L'output del lm_head è quindi una matrice di 5 vettori di 200064 (che costituiscono la dimensione del vocab).<br>
La probabilità del prossimo token è quindi l'ultimo di questi vettori

In [12]:
lm_head_output[0,-1]

tensor([26.1250, 30.0000, 23.2500,  ..., 11.0000, 11.0000, 11.0000],
       dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

Per selezionare il valore più alto (quindi quello con la massima probabilità) faccio:

In [13]:
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

' Paris'